In [4]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import random

from trainer.maskdataloader import CustomDataset

def plot_fbank(fbank, title=None, save_path=None):
    fig, axs = plt.subplots(1, 1)
    fig.set_figheight(4)
    fig.set_figwidth(8)
    axs.set_title(title or "Filter bank")
    cax1 = axs.imshow(fbank, aspect="auto", origin="lower", cmap='inferno')
    axs.set_ylabel("Frequency")
    axs.set_xlabel("Time")
    fig.colorbar(cax1, ax=axs, orientation='vertical', label='Intensity')
    if save_path:
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    

# read and save anomlaous spectrograms

In [6]:
patchsize= 4
maskratio=0.3


d_anomalies = CustomDataset([
    "data/evaluation/anomalies/broken_board/",
    "data/evaluation/anomalies/board_stuck/",
    "data/evaluation/anomalies/Uneven_thick_wood/",
], patch_size=(patchsize, patchsize), mask_ratio=maskratio, fixed_masking=True, return_directory=True)

print(len(d_anomalies))

####DATA LABELLING####
# save data and corresponding file names in csv
records = []

# Paths
save_img_dir = "saved_anomalous_spectrograms/"
os.makedirs(save_img_dir, exist_ok=True)
csv_path = os.path.join(save_img_dir, "anomaly_spectrogram_metadata.csv")

for idx in range(len(d_anomalies)):
    (_, (spec, _)), filepath = d_anomalies[idx]
    filename = os.path.basename(filepath)
    file_id = os.path.splitext(filename)[0]

    # Parse name parts
    parts = file_id.split("_")
    recording_name = "_".join(parts[:-1])
    spectrogram_number = int(parts[-1]) # convert to int for sorting and time calc

    # Derive board type from parent folder
    board_type = os.path.basename(os.path.dirname(filepath))

    # Map number to time range
    time_range = f"{(spectrogram_number - 1) * 10}-{spectrogram_number * 10} seconds"

    # Save spectrogram image
    image_path = os.path.join(save_img_dir, f"{file_id}.png")
    plot_fbank(spec.numpy(), title=f"{recording_name} #{spectrogram_number}", save_path=image_path)

    # Add row to CSV
    records.append({
        "Recording wav File": recording_name+".wav",
        "Spectrogram File Name": filename.replace(".pt", ""),
        "Spectrogram Number": str(spectrogram_number),
        "Spectrogram Time Range": time_range,
        "Board (Anomaly) Type": board_type,
        "Input Spectrogram": "",  # left empty for manual image insertion
        "Time Anomaly Labels": "", # left empty for manual labelling
        "Masked Labelled Spectrogram": "" # left empty for manual image insertion
    })

# Sort and save
df = pd.DataFrame(records)
df = df.sort_values(by=["Recording wav File", "Spectrogram Number"]).reset_index(drop=True)
df["Spectrogram Number"] = df["Spectrogram Number"].astype(str)
df.to_csv(csv_path, index=False)

print(f"Saved CSV to: {csv_path}")



105
Saved CSV to: saved_anomalous_spectrograms/anomaly_spectrogram_metadata.csv


In [7]:
print(filepath, len(df))

data/evaluation/anomalies/Uneven_thick_wood/MPD_19-12-04_1119_00157_2.pt 105
